In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import clear_output
import os
import pandapybullet as ppb
import scipy
import matplotlib

from robot import *
from utils import *
import pybullet as p
import pybullet_data
import networkx as nx
from functools import partial
%load_ext autoreload
%autoreload 2

## Setup Pybullet

In [ ]:
simulation_step = 1e-3
world_sim = ppb.WorldSim(gravity=[0,0,-10])
world_sim.set_time_step(simulation_step)

In [ ]:
p.resetSimulation()

In [ ]:
world_sim.setup_scene()

In [ ]:
robot_details = {
    'urdf': '/robots/panda/panda_arm_rivet.urdf',
    'base_position': [0, 0, 0],
    'base_orientation': [0, 0, 0],
    'ee_index': 11,
    'gripper_indices': [9, 10],
}

In [ ]:
robot = world_sim.add_robot(robot_details)

### Setup Obstacles

In [ ]:
is_simple = False
obs_ids = []
rgbacolor = (0,0,1,0)

board_id = p.loadURDF('urdf/support_cylindrical.urdf', globalScaling=0.0008, useFixedBase=1)
quat = w2quat(np.array([0,0,-np.pi/2]))
p.resetBasePositionAndOrientation(board_id, (0.6,-2.2,-.3), quat)

offset = 0.


In [ ]:
x_board = 0.47
p.resetBasePositionAndOrientation(board_id, (x_board,-1.7,-.41), quat)

### Setup targets in the middle

In [ ]:
ys = np.concatenate([np.arange(-0.34, -0.15, 0.05), np.arange(0.2, 0.36, 0.05)],-1)
zs = np.arange(0.35, 0.54, 0.05) + 0.2
x = x_board + 0.3
targets_mid = []
target_ids = []
offset = -0.05
for y in ys:
    for z in zs:
        target = np.array([x, y,z])
        targets_mid += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(1,0,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_mid[-1], (0,0,0,1))
        targets_mid[-1][0] -= 0.03

targets_mid = np.array(targets_mid)

### Targets on the bottom

In [ ]:
R = w2mat(quat2w(euler2quat([0,10,0.])))

In [ ]:
zs = np.arange(0.25,0.31, 0.05) + 0.2
#ys = np.array([-0.45, -0.25,  0.15,  0.35])
#zs = np.arange(0.6, 0.81, 0.1)
x = x_board + 0.25
targets_down = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_down += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,0,1,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_down[-1], (0,0,0,1))
        targets_down[-1] += (R@np.array([1,0,0]))*offset
        offset_x += 0.03

targets_down = np.array(targets_down)

In [ ]:
for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0 ], ornObj=euler2quat([0,10,0.]).tolist())

In [ ]:
robot.reset_q()

In [ ]:
p.addUserDebugLine(robot.x.tolist(), (targets_down[1]).tolist(), lineColorRGB=(1,0,0))

### Targets on the top

In [ ]:
R = w2mat(quat2w(euler2quat([0,-10,0.])))

In [ ]:
zs = np.arange(0.55,0.61, 0.05) + 0.2

x = x_board + 0.27
targets_up = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_up += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,1,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_up[-1], (0,0,0,1))
        targets_up[-1] += (R@np.array([1,0,0]))*offset
        offset_x -= 0.03

targets_up = np.array(targets_up)

In [ ]:
for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0], ornObj=euler2quat([0,-10,0.]).tolist())

In [ ]:
p.addUserDebugLine(robot.x.tolist(), (targets_up[0]).tolist(), lineColorRGB=(0,0,1))

### Try IK 

In [ ]:
robot.reset_q()


In [ ]:
qs_teguh = [[-0.83932275, -1.54137399,  0.95430008, -1.86467053,  2.40159777,
        3.59695137, -1.15764069],
            [-0.28033785, -1.75766024,  1.12916018, -2.29047562,  2.52327724,
        3.63650991, -0.64595397],
            [-0.31252869, -1.67061596,  0.78073781, -2.52464951,  2.34896703,
        2.82965887,  0.8981308 ],
            [ 0.57645143, -1.69599059,  0.31823516, -2.23416334,  2.39084771,
        3.50976051,  2.64189416],
            [ 0.47841517, -1.65205786, -0.48158265, -2.74690527,  1.4707326 ,
        3.33008487,  2.70119612],
            [ 0.53582313, -1.60804724,  0.23277705, -2.3715542 ,  2.16054946,
        3.6711258 ,  1.4526345 ]]


In [ ]:
np.save('qs_outside', np.stack(qs_teguh), allow_pickle=True)

In [ ]:
robot.q

In [ ]:
robot.q

In [ ]:
robot.teaching_mode()

In [ ]:
robot.rotation = "quat"

In [ ]:
p.resetBasePositionAndOrientation(board_id, (x_board,-1.7,-.41), quat)

In [ ]:
robot.ik(targets_down[2], orn_des=np.array([0,2.16,0.]) , K=2)

In [ ]:
euler2quat([0,2.16,0.])

In [ ]:
robot.orn

In [ ]:
robot.ik(targets_up[2], orn_des=np.array([0,1.029,0.]) , K=2)

In [ ]:
robot.ik(targets_mid[7], orn_des=np.array([0,np.pi/2,0]) , K=20)

In [ ]:
targets_x = {'up': targets_up,
               'mid': targets_mid,
               'down': targets_down}
targets_orn = {'up': euler2quat([0,1.029,0.]),
               'mid': euler2quat([0,np.pi/2,0]),
               'down': euler2quat([0,2.16,0.])}
R = {'up': w2mat(quat2w(euler2quat([0,10,0.]))),
     'mid':np.eye(3),
     'down':w2mat(quat2w(euler2quat([0,-10,0.])))
    }

In [ ]:
np.save('targets_x',targets_x, allow_pickle=True)

In [ ]:
np.save('targets_orn',targets_orn, allow_pickle=True)

In [ ]:
np.save('rotation_axes', R, allow_pickle=True)

#### check savings

In [ ]:
np.load('targets_orn.npy', allow_pickle=True)

### Check the joint positions after IK solution

In [ ]:
qs = np.load("good_q.npy", allow_pickle=True)

In [ ]:
hole_nb =49
sample = 0

In [ ]:
hole_nb += 1

In [ ]:
sample += 1
robot.set_q_(qs[hole_nb, sample])